In [1]:
import os
import pandas as pd
import numpy as np
import random
import json
import pickle
import re
from processer import Processer
from api import get_supplier, extract_data
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from gensim.sklearn_api.d2vmodel import D2VTransformer

In [2]:
train = pd.read_csv('train.txt')
train = train.set_index('nummer')
X = train["text"]
Y = train["lief2"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [3]:
train.head()

,Unnamed: 0,Unnamed: 0.1,sumbrutto,sumnetto,lief2,text
nummer,,,,,,
390486,4,4,0.00,0.00,"(158,1,0)",Yvonne Burri\nae ET eS eS a i a ee ee\n\nVon: ...
800045,25,25,350.00,350.00,"(158,1,0)",. SOIOUS\n« SOMAT\n\nMALA’ YSIA\n\n \n\ndriven...
800098,44,44,258.18,258.18,"(158,1,0)",\n\n \n\na a cS\n“BOMATEC\n: MALAYSIA\ndriven...
800099,43,43,126.00,126.00,"(158,1,0)",_ KIOOTN\nTEC:\n\n \n\n \n\nMALAYSIA\ndriven b...
800100,42,42,284.00,284.00,"(158,1,0)",_ .BOMATEC SCOOP\n\nMALAYSIA\n\ndriven by pass...


In [4]:
c = pickle.load(open('model.pkl', 'rb'))
pred = c.predict(X)
pred_proba = c.predict_proba(X)
print('F1-score:', f1_score(Y, pred, average='weighted'))

F1-score: 0.8743107331622405


In [43]:
idx = random.randint(0, len(Y))
print('Document no: %d' % Y.index[idx])
print('Predicted class: %s' % json.loads(get_supplier(pred[idx]))['name'])
print('Probability: %s' % np.amax(pred_proba[idx]))
print('True class: %s' % json.loads(get_supplier(Y.iloc[idx]))['name'])

Document no: 808836
Predicted class: Bomatec Management AG
Probability: 0.839697351690225
True class: Bomatec AG


In [51]:
print(X.loc[800116])

SOM ATEe SIME

 

 

a tis a
MALAYSIA
driven by passion

0 Jarul Ehsa INVOICE
Consignee : REFNO. : 00021209 DATE : 17-Nov-15
LAMPRECHT TRANSPORT LTD.
Building Z1,CH-8423 Embrach-Embraport FEEDER VESSEL :
Switzerland CMA CGM ALASKA V.0020W
Tel : +41 44 866 2133 MOTHER VESSEL :
Fax : +41 44 866 2112
Notify Party : FROM :PORTKLANG TO : ZURICH
BOMATEC AUTOMOTIVE AG
Hofstrasse 1, CH-8181 Hori, ETD :__ 21-Nov-15 ETA”—s:__26-Dec-15
Switzerland
TEL : +41-44-872 10 00 FREIGHT — : FOB PORT KLANG (COLLECT)

FAX: +41-44-872 1001
PAYMENT: BY T/T REMMITANCE

DESCRIPTION QUANTITY UNIT PRICE |
PCS @USD

“Good does not categorized as STA regulation"

   

MARKS & NOS

 
     
   
  
    

  

Pls refer to Electronic Component Of Magnet
Packing List HS Code : 8505.19.000

   
  

1) : Mr. M. Aulfinger

    
    
    
 
   
   
   
   
   
   
   
   
 
  

 

  
 
      
 
   
   
   

1.1] Part Name : Aimant Plat 57,555 2,423.07
Part No : 7217P078.007
Position No : 001
Pos Ref No : 20001121800001
Artic

In [38]:
folder_path = "./data/PI20_" + str(Y.index[idx])
file_name = os.listdir(os.getcwd() + folder_path)[0]
os.chdir(folder_path)
os.startfile(file_name)
os.chdir('../..')

In [47]:
print(extract_data(pred[idx], X.iloc[idx]))

{'invoice_number': '10-411374', 'amount': '46.48', 'invoice_date': '21.08.2019', 'currency': 'CHF'}


In [45]:
train.iloc[idx]

Unnamed: 0                                                    192
Unnamed: 0.1                                                  192
sumbrutto                                                   46.48
sumnetto                                                    43.16
lief2                                                   (154,1,0)
text            SOMATEC:\n\nCOE? ZS\n\nmore than just magnets\...
Name: 808836, dtype: object